In [2]:
# + tags=[]
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import box, Point
%matplotlib inline

In [3]:
# + tags=[]
csv_path = "data/raw/ais/aisdk-2024-03-01.csv"
# Wir lesen nur die ersten 100 000 Zeilen zum schnellen Test
df = pd.read_csv(csv_path, parse_dates=['# Timestamp'], nrows=100_000)
print("Spalten:", df.columns.tolist())
df.head()

Spalten: ['# Timestamp', 'Type of mobile', 'MMSI', 'Latitude', 'Longitude', 'Navigational status', 'ROT', 'SOG', 'COG', 'Heading', 'IMO', 'Callsign', 'Name', 'Ship type', 'Cargo type', 'Width', 'Length', 'Type of position fixing device', 'Draught', 'Destination', 'ETA', 'Data source type', 'A', 'B', 'C', 'D']


,# Timestamp,Type of mobile,MMSI,Latitude,Longitude,Navigational status,ROT,SOG,COG,Heading,...,Length,Type of position fixing device,Draught,Destination,ETA,Data source type,A,B,C,D
0,2024-01-03,Class A,219000873,56.990910,10.304543,Under way using engine,NaN,0.0,30.2,NaN,...,NaN,Undefined,NaN,Unknown,NaN,AIS,NaN,NaN,NaN,NaN
1,2024-01-03,Base Station,2190068,56.447260,10.945872,Unknown value,NaN,NaN,NaN,NaN,...,NaN,GPS,NaN,Unknown,NaN,AIS,NaN,NaN,NaN,NaN
2,2024-01-03,Class A,219016683,56.800165,9.024933,Under way using engine,0.0,0.0,257.3,17.0,...,NaN,Undefined,NaN,Unknown,NaN,AIS,NaN,NaN,NaN,NaN
3,2024-01-03,Class A,219000615,56.967093,9.224287,Restricted maneuverability,0.0,2.3,69.8,67.0,...,NaN,Undefined,NaN,Unknown,NaN,AIS,NaN,NaN,NaN,NaN
4,2024-01-03,Base Station,2190071,57.110043,8.648282,Unknown value,NaN,NaN,NaN,NaN,...,NaN,GPS,NaN,Unknown,NaN,AIS,NaN,NaN,NaN,NaN


In [44]:
# + tags=[]
import pandas as pd
import geopandas as gpd
from shapely.geometry import box

# 1) BBox aus GeoJSON (vom Plotseamap-Buffer) laden und zu einem einzigen Polygon vereinigen
bbox_gdf = (
    gpd.read_file("data/processed/geojson/fehmarnbelt_buffer.geojson")
       .to_crs("EPSG:4326")
)
clip_poly = bbox_gdf.geometry.union_all()  # ein Shapely-Polygon

# 2) DataFrame mit AIS-Daten (erste 100k Zeilen zum Test)
df = pd.read_csv("data/raw/ais/aisdk-2024-03-01.csv",
                 parse_dates=['# Timestamp'])

# 3) Spalten umbenennen
df = df.rename(columns={'Latitude':'lat', 'Longitude':'lon'})

# 4) GeoDataFrame erzeugen
gdf = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df['lon'], df['lat']),
    crs="EPSG:4326"
)

# 5) Clip: Maske auf clip_poly (kein Alignment mehr nötig!)
mask = gdf.geometry.within(clip_poly)
gdf = gdf.loc[mask]

KeyboardInterrupt: 

In [18]:
gdf.head()

,# Timestamp,Type of mobile,MMSI,lat,lon,Navigational status,ROT,SOG,COG,Heading,...,Type of position fixing device,Draught,Destination,ETA,Data source type,A,B,C,D,geometry
127,2024-01-03 00:00:00,AtoN,992111745,54.554347,11.291427,Unknown value,NaN,NaN,NaN,NaN,...,GPS,NaN,Unknown,NaN,AIS,NaN,1.0,NaN,1.0,POINT (11.29143 54.55435)
135,2024-01-03 00:00:00,Class A,245439000,54.499727,11.241043,Under way using engine,-7.5,0.0,234.0,319.0,...,Undefined,NaN,Unknown,NaN,AIS,NaN,NaN,NaN,NaN,POINT (11.24104 54.49973)
182,2024-01-03 00:00:00,Class A,211190000,54.653467,11.351433,Under way using engine,0.0,0.0,141.5,49.0,...,Undefined,NaN,Unknown,NaN,AIS,NaN,NaN,NaN,NaN,POINT (11.35143 54.65347)
193,2024-01-03 00:00:01,Class A,211190000,54.653467,11.351433,Under way using engine,0.0,0.0,141.5,49.0,...,Undefined,NaN,Unknown,NaN,AIS,NaN,NaN,NaN,NaN,POINT (11.35143 54.65347)
194,2024-01-03 00:00:01,Class A,211190000,54.653467,11.351433,Under way using engine,0.0,0.0,141.5,49.0,...,Undefined,NaN,Unknown,NaN,AIS,NaN,NaN,NaN,NaN,POINT (11.35143 54.65347)


In [42]:
# 5) Dynamisches Filtern anhand der Config
# 1) Config laden
import json
with open("config/ais_filters.json") as f:
    cfg = json.load(f)

for col, crit in cfg["ais_filters"].items():
    if isinstance(crit, list):
        gdf = gdf[gdf[col].isin(crit)]
    else:
        gdf = gdf[gdf[col] == crit]

# 6) Reproject für Plot
gdf = gdf.to_crs(epsg=3857)
print(f"Nach Config-Filter: {len(gdf)} Punkte")
gdf.head()

Nach Config-Filter: 205 Punkte


,# Timestamp,Type of mobile,MMSI,lat,lon,Navigational status,ROT,SOG,COG,Heading,...,Type of position fixing device,Draught,Destination,ETA,Data source type,A,B,C,D,geometry
1271,2024-01-03 00:00:07,Class A,212198000,54.567695,11.229103,Under way using engine,0.0,7.5,120.1,121.0,...,GPS,3.8,GDANSK,02/03/2024 13:00:00,AIS,72.0,10.0,6.0,7.0,POINT (1250018.028 7278412.565)
1274,2024-01-03 00:00:07,Class A,212198000,54.567695,11.229103,Under way using engine,0.0,7.5,120.1,121.0,...,GPS,3.8,GDANSK,02/03/2024 13:00:00,AIS,72.0,10.0,6.0,7.0,POINT (1250018.028 7278412.565)
1277,2024-01-03 00:00:07,Class A,212198000,54.567695,11.229103,Under way using engine,0.0,7.5,120.1,121.0,...,GPS,3.8,GDANSK,02/03/2024 13:00:00,AIS,72.0,10.0,6.0,7.0,POINT (1250018.028 7278412.565)
2884,2024-01-03 00:00:17,Class A,212198000,54.567522,11.229615,Under way using engine,0.0,7.4,120.1,121.0,...,GPS,3.8,GDANSK,02/03/2024 13:00:00,AIS,72.0,10.0,6.0,7.0,POINT (1250075.024 7278379.346)
6249,2024-01-03 00:00:37,Class A,212198000,54.567177,11.230647,Under way using engine,0.0,7.4,119.5,121.0,...,GPS,3.8,GDANSK,02/03/2024 13:00:00,AIS,72.0,10.0,6.0,7.0,POINT (1250189.905 7278313.101)


In [39]:
filtered["Cargo type"]=="Category X"

0

In [ ]:
data/processed/ais/fehmarnbelt_ais_filtered.csv

In [45]:
# + tags=[]
csv_path = "data/processed/ais/fehmarnbelt_ais_filtered.csv"
# Wir lesen nur die ersten 100 000 Zeilen zum schnellen Test
df = pd.read_csv(csv_path, parse_dates=['# Timestamp'])
print("Spalten:", df.columns.tolist())
df.head()

Spalten: ['# Timestamp', 'Type of mobile', 'MMSI', 'lat', 'lon', 'Navigational status', 'ROT', 'SOG', 'COG', 'Heading', 'IMO', 'Callsign', 'Name', 'Ship type', 'Cargo type', 'Width', 'Length', 'Type of position fixing device', 'Draught', 'Destination', 'ETA', 'Data source type', 'A', 'B', 'C', 'D']


,# Timestamp,Type of mobile,MMSI,lat,lon,Navigational status,ROT,SOG,COG,Heading,...,Length,Type of position fixing device,Draught,Destination,ETA,Data source type,A,B,C,D
0,2024-01-03 00:00:07,Class A,212198000,54.567695,11.229103,Under way using engine,0.0,7.5,120.1,121.0,...,82.0,GPS,3.8,GDANSK,02/03/2024 13:00:00,AIS,72.0,10.0,6.0,7.0
1,2024-01-03 00:00:07,Class A,212198000,54.567695,11.229103,Under way using engine,0.0,7.5,120.1,121.0,...,82.0,GPS,3.8,GDANSK,02/03/2024 13:00:00,AIS,72.0,10.0,6.0,7.0
2,2024-01-03 00:00:07,Class A,212198000,54.567695,11.229103,Under way using engine,0.0,7.5,120.1,121.0,...,82.0,GPS,3.8,GDANSK,02/03/2024 13:00:00,AIS,72.0,10.0,6.0,7.0
3,2024-01-03 00:00:17,Class A,212198000,54.567522,11.229615,Under way using engine,0.0,7.4,120.1,121.0,...,82.0,GPS,3.8,GDANSK,02/03/2024 13:00:00,AIS,72.0,10.0,6.0,7.0
4,2024-01-03 00:00:37,Class A,212198000,54.567177,11.230647,Under way using engine,0.0,7.4,119.5,121.0,...,82.0,GPS,3.8,GDANSK,02/03/2024 13:00:00,AIS,72.0,10.0,6.0,7.0
